# Hybrid Recommender using Deep Neural Network

*Side note:* <br>
Tensorflow version is 1.15. <br>
*Lazy* approach. Hard to maintain but efficient.


In [1]:
import pandas as pd
import numpy as np

In [2]:
!gdown --id '1Y8MIETuWdWOGWAwkg65gps2est1OHa6P'

Downloading...
From: https://drive.google.com/uc?id=1Y8MIETuWdWOGWAwkg65gps2est1OHa6P
To: /content/Data_1 (TF-IDF + Little_Listing_Features + SVD).csv
195MB [00:01, 106MB/s]


In [3]:
df_cross = pd.read_csv('/content/Data_1 (TF-IDF + Little_Listing_Features + SVD).csv').iloc[:,1:]

In [4]:
cols = df_cross.columns.to_list()

cols = map(lambda x:x.replace(' ','_'),cols)

In [5]:
df_cross.columns = cols

## Data Preparation

In [ ]:
!pip install tensorflow==1.15
import os
import tensorflow as tf
import tensorflow_hub as hub

In [6]:
# Determine csv and label columns
num_factor = 10
text_feat = 100

NON_FACTOR_COLUMNS = ["preference",'listing_id',
       'host_is_superhost', 'latitude',
       'longitude','price','number_of_reviews_ltm','calculated_host_listings_count','nearby_sites_rating',
       'nearest_station_dist', 'host_response_time_cross_rate']
FACTOR_COLUMNS1 = ["user_factor_{}".format(i) for i in range(num_factor)] + ["item_factor_{}".format(i) for i in range(num_factor)]
FACTOR_COLUMNS2 = ["text_factor_{}".format(i) for i in range(text_feat)] 
CSV_COLUMNS = NON_FACTOR_COLUMNS + FACTOR_COLUMNS1 + FACTOR_COLUMNS2
LABEL_COLUMN = "preference"

# Set default values for each CSV column
NON_FACTOR_DEFAULTS = [[0.0],['Unknown'],[0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]]
FACTOR_DEFAULTS1 = [[0.0] for i in range(num_factor)] + [[0.0] for i in range(num_factor)] # user and item
FACTOR_DEFAULTS2 = [[0.0] for i in range(text_feat)]
DEFAULTS = NON_FACTOR_DEFAULTS + FACTOR_DEFAULTS1 + FACTOR_DEFAULTS2

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest = train_test_split(df_cross, test_size=0.2, random_state=1)
Xtrain, Xval = train_test_split(Xtrain, test_size=0.2, random_state=1)
print(f"Shape of train data: {Xtrain.shape}")
print(f"Shape of validation data: {Xval.shape}")
print(f"Shape of test data: {Xtest.shape}")

Xtrain.to_csv('train.csv',header=False,index=False)
Xval.to_csv('val.csv',header=False,index=False)
Xtest.to_csv('test.csv',header=False,index=False)

Shape of train data: (76640, 132)
Shape of validation data: (19161, 132)
Shape of test data: (23951, 132)


### Make auxiliary functions

In [ ]:
lat_min = df_cross.latitude.min()
lat_max = df_cross.latitude.max()

lon_min = df_cross.longitude.min()
lon_max = df_cross.longitude.max()

norl_mean = df_cross.number_of_reviews_ltm.mean()
norl_std = df_cross.number_of_reviews_ltm.std()

chlc_mean = df_cross.calculated_host_listings_count.mean()
chlc_std = df_cross.calculated_host_listings_count.std()

nsr_mean = df_cross.nearby_sites_rating.mean()
nsr_std = df_cross.nearby_sites_rating.std()

nsd_mean = df_cross.nearest_station_dist.mean()
nsd_std = df_cross.nearest_station_dist.std()

In [ ]:
# Create input function for train and eval
def read_dataset(filename, mode, batch_size = 512):
    def _input_fn(): 
        def decode_csv(value_column):
            columns = tf.decode_csv(records = value_column, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))          
            label = features.pop(LABEL_COLUMN)         
            return features, label

        # Create list of files that match pattern
        file_list = tf.gfile.Glob(filename = filename)

        # Create dataset from file list
        dataset = tf.data.TextLineDataset(filenames = file_list).map(map_func = decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(count = num_epochs).batch(batch_size = batch_size)
        res = dataset.make_one_shot_iterator().get_next()

        # if res.keys() in df_cross.columns:
        #   return dataset.make_one_shot_iterator().get_next()
        # else:
        return res
    return _input_fn

In [ ]:
num_listing_embedding = 100
#num_reviewer_embedding = 100

# Create feature columns to be used in model
def create_feature_columns():
    # Create listing id feature column
    listing_id_column = tf.feature_column.categorical_column_with_hash_bucket(
        key = "listing_id",
        hash_bucket_size = number_of_listing_id)
    
    embedded_listing_id_column = tf.feature_column.embedding_column(
        categorical_column = listing_id_column,
        dimension = num_listing_embedding)    

    # Create reviewer id feature column
    # reviewer_id_column = tf.feature_column.categorical_column_with_hash_bucket(
    #     key = "reviewer_id",
    #     hash_bucket_size = number_of_reviewer_id)
    
    # embedded_reviewer_id_column = tf.feature_column.embedding_column(
    #     categorical_column = reviewer_id_column,
    #     dimension = num_reviewer_embedding)    

    # Create superhost feature column
    categorical_superhost_column = tf.feature_column.categorical_column_with_identity(
        key = 'host_is_superhost',
        num_buckets = 2)
    
    indicator_superhost_column = tf.feature_column.indicator_column(
        categorical_column = categorical_superhost_column
    )

    # Create bucketized price column
    bucketized_price_column = tf.feature_column.bucketized_column(
        tf.feature_column.numeric_column(key = 'price'),
        boundaries=list(np.arange(0,1000,100)))

    # Create numeric columns: 'number_of_reviews_ltm','calculated_host_listings_count','nearby_sites_rating',
    #'nearest_station_dist','host_response_time_cross_rate'

    number_of_reviews_ltm_column = tf.feature_column.numeric_column(
        key = 'number_of_reviews_ltm')#, 
        #normalizer_fn=lambda x:(x-norl_mean) / norl_std)
    
    calculated_host_listings_count_column = tf.feature_column.numeric_column(
        key = 'calculated_host_listings_count')
        #,normalizer_fn=lambda x:(x-chlc_mean) / chlc_std)
    
    nearby_sites_rating_column = tf.feature_column.numeric_column(
        key = 'nearby_sites_rating')
        #,normalizer_fn=lambda x:(x-nsr_mean) / nsr_std)
    
    nearest_station_dist_column = tf.feature_column.numeric_column(
        key = 'nearest_station_dist')
        #,normalizer_fn=lambda x:(x-nsd_mean) / nsd_std)

    host_response_time_cross_rate_column = tf.feature_column.numeric_column(
        key = 'host_response_time_cross_rate')
  

    # Create lat,lon boundaries list for our binning
    lat_boundaries = list(np.arange(lat_min, lat_max+0.01, 0.05))
    lon_boundaries = list(np.arange(lon_min, lon_max+0.01, 0.05))

    lat_column = tf.feature_column.numeric_column(
        key = "latitude")
    lon_column = tf.feature_column.numeric_column(
        key = "longitude")   
    
    # Create bucketized lat,lon feature column using our boundaries
    lat_bucketized_column = tf.feature_column.bucketized_column(
        source_column = lat_column,
        boundaries = lat_boundaries)
    
    lon_bucketized_column = tf.feature_column.bucketized_column(
        source_column = lon_column,
        boundaries = lon_boundaries)    
    
    # Cross bucketized lat column and bucketized lon column
    crossed_lat_lon_column = tf.feature_column.crossed_column(
        keys = [lat_bucketized_column, lon_bucketized_column],
        hash_bucket_size = len(lat_bucketized_column) * len(lon_bucketized_column))

    # Convert crossed categorical category and bucketized months since epoch column into indicator column so that it can be used in a DNN
    indicator_crossed_lat_lon_column = tf.feature_column.indicator_column(
            categorical_column = crossed_lat_lon_column)    

    # Create user and item factor feature columns from our trained WALS model
    user_factors = [tf.feature_column.numeric_column(key = "user_factor_" + str(i)) for i in range(num_factor)]
    item_factors =  [tf.feature_column.numeric_column(key = "item_factor_" + str(i)) for i in range(num_factor)]
    text_factors = [tf.feature_column.numeric_column(key = "text_factor_" + str(i)) for i in range(text_feat)]

    # Create list of feature columns
    feature_columns = [
      embedded_listing_id_column,    
      # embedded_reviewer_id_column,       
      indicator_superhost_column,
      bucketized_price_column,
      calculated_host_listings_count_column,
      nearby_sites_rating_column,
      nearest_station_dist_column,
      host_response_time_cross_rate_column,
      indicator_crossed_lat_lon_column] + user_factors + item_factors + text_factors

    return feature_columns

In [ ]:
# Create custom model function for our custom estimator
def model_fn(features, labels, mode, params):
    # Create neural network input layer using our feature columns defined above
    net = tf.feature_column.input_layer(features = features, feature_columns = params["feature_columns"])

    # Create hidden layers by looping through hidden unit list
    for units in params["hidden_units"]:
        net = tf.nn.dropout(
          net, params['dropout_rate'], seed=1
        )
        net = tf.layers.dense(inputs = net, 
                              units = units, 
                              activation = tf.nn.relu,
                              kernel_regularizer = tf.keras.regularizers.l1_l2(l1=params["l1"],l2=params["l2"]))

    x_out = tf.layers.Dense(1, activation=tf.nn.sigmoid)(net)

    # If the mode is prediction
    if mode == tf.estimator.ModeKeys.PREDICT:
        # Create predictions dict
        predictions_dict = {
            "preference": tf.expand_dims(input = x_out, axis = -1),
        }

        # Create export outputs
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = predictions_dict)}

        return tf.estimator.EstimatorSpec( # return early since we"re done with what we need for prediction mode
          mode = mode,
          predictions = predictions_dict,
          loss = None,
          train_op = None,
          eval_metric_ops = None,
          export_outputs = export_outputs)


    # Compute loss using sparse softmax cross entropy since this is classification and our labels (content id indices) and probabilities are mutually exclusive
    loss = tf.losses.mean_squared_error(labels, x_out[0])

    # If the mode is evaluation
    if mode == tf.estimator.ModeKeys.EVAL:
        # Metrics
        rmse = tf.metrics.root_mean_squared_error(labels, x_out)
        mae = tf.metrics.mean_absolute_error(labels, x_out)

        # Put eval metrics into a dictionary
        eval_metric_ops = {
            "RMSE": rmse,
            "MAE": mae}

        # Create scalar summaries to see in TensorBoard
        tf.summary.scalar(name = "RMSE", tensor = rmse[1])
        tf.summary.scalar(name = "MAE", tensor = mae[1])   
        
        return tf.estimator.EstimatorSpec( # return early since we"re done with what we need for evaluation mode
            mode = mode,
            predictions = None,
            loss = loss,
            train_op = None,
            eval_metric_ops = eval_metric_ops,
            export_outputs = None)

    # Continue on with training mode

    # If the mode is training
    assert mode == tf.estimator.ModeKeys.TRAIN

    # Create a custom optimizer
    optimizer = tf.train.AdagradOptimizer(learning_rate = params["learning_rate"])

    # Create train op
    train_op = optimizer.minimize(loss = loss, global_step = tf.train.get_global_step())

    return tf.estimator.EstimatorSpec( # final return since we"re done with what we need for training mode
        mode = mode,
        predictions = None,
        loss = loss,
        train_op = train_op,
        eval_metric_ops = None,
        export_outputs = None)

In [ ]:
# Create serving input function
def serving_input_fn():  
    feature_placeholders = {
        colname : tf.placeholder(dtype = tf.float64, shape = [None]) \
        for colname in NON_FACTOR_COLUMNS[3:]
    }

 #   feature_placeholders[NON_FACTOR_COLUMNS[0]] = tf.placeholder(dtype = tf.string, shape = [None])
    feature_placeholders[NON_FACTOR_COLUMNS[0]] = tf.placeholder(dtype = tf.float64, shape = [None])
    feature_placeholders[NON_FACTOR_COLUMNS[1]] = tf.placeholder(dtype = tf.string, shape = [None])
    feature_placeholders[NON_FACTOR_COLUMNS[2]] = tf.placeholder(dtype = tf.int64, shape = [None])

    for colname in FACTOR_COLUMNS1 + FACTOR_COLUMNS2:
        feature_placeholders[colname] = tf.placeholder(dtype = tf.float64, shape = [None])

    features = {
        key: tf.expand_dims(tensor, -1) \
        for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

### Train, evaluate and predict

In [ ]:
# Create train and evaluate loop to combine all of the pieces together.
tf.logging.set_verbosity(tf.logging.INFO)
def train_and_evaluate(args):
    estimator = tf.estimator.Estimator(
        model_fn = model_fn,
        model_dir = args["output_dir"],
        params = {
        "feature_columns": create_feature_columns(),
        "hidden_units": args["hidden_units"],
        "learning_rate": args["learning_rate"],
        "dropout_rate": args["dropout_rate"],
        "l1": args["l1"],
        "l2": args["l2"]
        }
    )

    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset(filename = args["train_data_paths"], mode = tf.estimator.ModeKeys.TRAIN, batch_size = args["batch_size"]),
        max_steps = args["train_steps"])

    exporter = tf.estimator.LatestExporter(name = "exporter", 
                                           serving_input_receiver_fn = serving_input_fn)

    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset(filename = args["eval_data_paths"], mode = tf.estimator.ModeKeys.EVAL, batch_size = args["batch_size"]),
        steps = None,
        start_delay_secs = args["start_delay_secs"],
        throttle_secs = args["throttle_secs"],
        exporters = exporter)

    tf.estimator.train_and_evaluate(estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)
    return estimator

In [ ]:
# Call train and evaluate loop
import shutil

outdir = "hybrid_recommendation_trained"
shutil.rmtree(path = outdir, ignore_errors = True) # start fresh each time

arguments = {
    "train_data_paths": "train.csv",
    "eval_data_paths": "test.csv",
    "output_dir": outdir,
    "batch_size": 128,
    "learning_rate": 0.1,
    "dropout_rate": 0.75,
    "l2": 0.01,
    "l1": 0.01,
    "hidden_units": [512, 256, 128, 64, 32],
    "train_steps": 1000,
    "start_delay_secs": 30,
    "throttle_secs": 30
}

model = train_and_evaluate(arguments)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'hybrid_recommendation_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa725df06d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordi

In [ ]:
test_pred_raw = [x['preference'] for x in list(model.predict(input_fn=read_dataset('test.csv',mode=tf.estimator.ModeKeys.PREDICT)))]
test_pred = [x[0][0] for x in test_pred_raw]
testset_mse = sum(np.square(Xtest.preference - test_pred))/len(test_pred)
testset_mse

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from hybrid_recommendation_trained/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.03931196245572145

In [ ]:
export_dir = '/content/exported_model/1'
model.export_saved_model(export_dir,serving_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Signatures INCLUDED in export for Classify: None


...

INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /content/exported_model/4/temp-b'1617604220'/saved_model.pb


INFO:tensorflow:SavedModel written to: /content/exported_model/4/temp-b'1617604220'/saved_model.pb


b'/content/exported_model/4/1617604220'

In [ ]:
from tensorflow.contrib import predictor

predict_fn = predictor.from_saved_model('/content/exported_model/4/1617604220')
predictions = predict_fn(Xtest.astype({'listing_id':'string','number_of_reviews_ltm':'float64','calculated_host_listings_count':'float64'}))
print(predictions)

INFO:tensorflow:Restoring parameters from /content/exported_model/4/1617604220/variables/variables


INFO:tensorflow:Restoring parameters from /content/exported_model/4/1617604220/variables/variables


{'preference': array([[[0.88810277]],

       [[0.794929  ]],

       [[0.8260367 ]],

       ...,

       [[0.8244647 ]],

       [[0.8540371 ]],

       [[0.8230056 ]]], dtype=float32)}
